In [1]:
from safetensors.torch import load_file
import torch

path = "/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia/checkpoint-5000/model.safetensors"
weights = load_file(path)

print("✅ 전체 키 목록:")
for k in weights.keys():
    print(k)

print("\n✅ 주요 가중치 분석:")
combine_weight = weights.get("embed_out.combine.weight", None)
if combine_weight is not None:
    print("🧩 combine.weight shape:", combine_weight.shape)
    print("📈 mean:", combine_weight.mean().item())
    print("📉 std:", combine_weight.std().item())
    print("✅ norm:", combine_weight.norm().item())
else:
    print("❌ combine.weight 없음")

# gripper head 쪽 확인
if "embed_out.action_head.weight" in weights:
    gripper_head = weights["embed_out.action_head.weight"][-1]  # 마지막 차원이 gripper라고 가정
    print("🤖 gripper head mean/std:", gripper_head.mean().item(), "/", gripper_head.std().item())


✅ 전체 키 목록:
embed_out.combine.bias
embed_out.combine.weight
embed_out.diffusion_step_encoder.1.bias
embed_out.diffusion_step_encoder.1.weight
embed_out.diffusion_step_encoder.3.bias
embed_out.diffusion_step_encoder.3.weight
embed_out.down_modules.0.0.blocks.0.block.0.bias
embed_out.down_modules.0.0.blocks.0.block.0.weight
embed_out.down_modules.0.0.blocks.0.block.1.bias
embed_out.down_modules.0.0.blocks.0.block.1.weight
embed_out.down_modules.0.0.blocks.1.block.0.bias
embed_out.down_modules.0.0.blocks.1.block.0.weight
embed_out.down_modules.0.0.blocks.1.block.1.bias
embed_out.down_modules.0.0.blocks.1.block.1.weight
embed_out.down_modules.0.0.cond_encoder.1.bias
embed_out.down_modules.0.0.cond_encoder.1.weight
embed_out.down_modules.0.0.residual_conv.bias
embed_out.down_modules.0.0.residual_conv.weight
embed_out.down_modules.0.1.blocks.0.block.0.bias
embed_out.down_modules.0.1.blocks.0.block.0.weight
embed_out.down_modules.0.1.blocks.0.block.1.bias
embed_out.down_modules.0.1.blocks.0.bl

In [3]:
import pickle

def load_pkl(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

# 사용 예시
file_path = "/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia/dataset_stats.pkl"
data = load_pkl(file_path)

# 상위 키 출력
if isinstance(data, dict):
    print("🔑 Top-level keys:", list(data.keys())[:10])

# 예시로 일부 값 출력
for key in list(data.keys())[:2]:
    print(f"\n📌 [{key}] 값:")
    print(data[key])


🔑 Top-level keys: ['action_mean', 'action_std', 'action_min', 'action_max', 'qpos_mean', 'qpos_std', 'example_qpos']

📌 [action_mean] 값:
[ 0.5505599   0.015884    0.28063768  0.768541   -0.02702801 -0.06607575
 -0.03195749 -0.79086196 -0.02463053  0.34631187]

📌 [action_std] 값:
[0.11837748 0.17448317 0.15690881 0.34308028 0.46564147 0.26412168
 0.4723583  0.3456244  0.17425872 0.41214043]


In [4]:
import pickle

with open("/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia/dataset_stats.pkl", "rb") as f:
    stats = pickle.load(f)

print("qpos_mean shape:", stats["qpos_mean"].shape)
print("qpos_mean:", stats["qpos_mean"])


qpos_mean shape: (7,)
qpos_mean: [ 0.54772806  0.01526346  0.27889764  0.4975223  -0.06598232  0.03752687
  0.30774996]


In [ ]:
# 역정규화에 필요한 값
ACTION_MEAN = torch.tensor([
    0.5505599, 0.015884, 0.28063768, 0.768541, -0.02702801,
    -0.06607575, -0.03195749, -0.79086196, -0.02463053, 0.34631187
]).cuda()

ACTION_STD = torch.tensor([
    0.11837748, 0.17448317, 0.15690881, 0.34308028, 0.46564147,
    0.26412168, 0.4723583, 0.3456244, 0.17425872, 0.41214043
]).cuda()


In [23]:
import h5py

h5_path = "/home/parkjeongsu/TinyVLA/droid_with_lang/droid_1dot7t_lang_succ_t0001_s-0-0/episode_1225616.hdf5"

with h5py.File(h5_path, 'r') as f:
    print("🔑 Top-level keys:", list(f.keys()))

    obs = f["observations"]
    print("🔍 [observations] 내부 key 목록:", list(obs.keys()))

    for k in obs.keys():
        item = obs[k]
        if isinstance(item, h5py.Dataset):
            print(f"✅ {k} shape:", item.shape)
        elif isinstance(item, h5py.Group):
            print(f"📁 {k} is a Group → 내부 key 목록:", list(item.keys()))

with h5py.File('/home/parkjeongsu/TinyVLA/droid_with_lang/droid_1dot7t_lang_succ_t0001_s-0-0/episode_1225616.hdf5', 'r') as f:
    print(f['action'].shape)  # ← 이게 바로 10차원이 되어야 함



🔑 Top-level keys: ['action', 'is_edited', 'language_raw', 'observations']
🔍 [observations] 내부 key 목록: ['images', 'qpos', 'qvel']
📁 images is a Group → 내부 key 목록: ['left', 'right']
✅ qpos shape: (189, 7)
✅ qvel shape: (189, 7)
(189, 10)


In [22]:
import json

# JSON 파일 경로
json_path = "/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia/special_tokens_map.json"

with open(json_path, "r") as f:
    data = json.load(f)

print(f"📌 data 타입: {type(data)}")

if isinstance(data, dict):
    for key in data:
        print(f"\n🔑 key: {key}")
        print(f"  🔹 value type: {type(data[key])}")
        if isinstance(data[key], list):
            print(f"  📏 list 길이: {len(data[key])}")
            if len(data[key]) > 0:
                print(f"  🔍 첫 번째 요소 타입: {type(data[key][0])}")
                print(f"  🧾 첫 번째 요소 내용: {data[key][0]}")
elif isinstance(data, list):
    print(f"🔹 list 길이: {len(data)}")
    print(f"🔍 첫 번째 요소 타입: {type(data[0])}")
    print(f"🧾 첫 번째 요소 내용: {data[0]}")
else:
    print("❌ 예상치 못한 데이터 타입입니다.")


📌 data 타입: <class 'dict'>

🔑 key: bos_token
  🔹 value type: <class 'dict'>

🔑 key: eos_token
  🔹 value type: <class 'dict'>

🔑 key: pad_token
  🔹 value type: <class 'str'>

🔑 key: unk_token
  🔹 value type: <class 'dict'>
